#### Project Goals

The goal of the project is to explore a datset of Jeopardy questions and answers and create some funcitons that allows us to study the answers and derive insights.

In [11]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

Getting the Data Ready. One of the things we needed to update were the titles of each column. The spaces in the front of each column needed to be removed in order to make working with the data frame a lot easier.

In [16]:
jeopardy_file = 'jeopardy.csv'

jeopardy_data = pd.read_csv(jeopardy_file)

#The titles of the columns contain a space in font of them, the code below removes the space.
jeopardy_data.rename(columns={' Show Number': 'Show Number', 
                              ' Air Date': 'Air Date',
                              ' Round': 'Round', 
                              ' Category': 'Category',
                              ' Value': 'Value', 
                              ' Question': 'Question',
                              ' Answer': 'Answer'}, inplace=True)

#Resetting Index
jeopardy_data.reset_index(drop=True, inplace=True)

#Dropping potential duplicate columns
jeopardy_data.drop_duplicates()

print(jeopardy_data)

        Show Number    Air Date             Round  \
0              4680  2004-12-31         Jeopardy!   
1              4680  2004-12-31         Jeopardy!   
2              4680  2004-12-31         Jeopardy!   
3              4680  2004-12-31         Jeopardy!   
4              4680  2004-12-31         Jeopardy!   
...             ...         ...               ...   
216925         4999  2006-05-11  Double Jeopardy!   
216926         4999  2006-05-11  Double Jeopardy!   
216927         4999  2006-05-11  Double Jeopardy!   
216928         4999  2006-05-11  Double Jeopardy!   
216929         4999  2006-05-11   Final Jeopardy!   

                               Category  Value  \
0                               HISTORY   $200   
1       ESPN's TOP 10 ALL-TIME ATHLETES   $200   
2           EVERYBODY TALKS ABOUT IT...   $200   
3                      THE COMPANY LINE   $200   
4                   EPITAPHS & TRIBUTES   $200   
...                                 ...    ...   
216925       

Function that calls a list that searches the dataset questions for any key words you choose. This funciton will allow us to call questions from a specific area of study. 

In [19]:
def filter_data(data, words):
  filter = lambda x: all(word in x for word in words)
  return data.loc[data["Question"].apply(filter)]


filtered = filter_data(jeopardy_data, ["Beatles", "American"])
print(filtered["Question"])

191857    Take 2 letters off a Beatles song title & you get this title of Paul McCartney's 2002 live CD of his American tour
Name: Question, dtype: object


This code allows us to explore the difficulty of groups of questions by using the Value column. The Value column denotes the difficutly by the price range. (The higher the price the higher the difficulty) A hurdle that I encountered in the function where the characters and None values within the column. Since, the column is set up as strings and not integers - exploratory analysis proofed difficult. Converting the string to intergers also proofed a bit of a challenge since I needed to convert string to integers while also removing special characters while keepiong the Nan values in mind in the function.

In [20]:
# Adding a new column. If the value of the float column is not "None", then we cut off the first character (which is a dollar sign), and replace all commas with nothing, and then cast that value to a float. If the answer was "None", then we just enter a 0.

def convert_to_float(value):
    try:
        return float(str(value).replace('$', '').replace(',', ''))
    except (ValueError, TypeError):
        return 0  # or any other default value to handle non-convertible cases

jeopardy_data["Float Value"] = jeopardy_data["Value"].apply(lambda x: convert_to_float(x))

# Filtering the dataset and finding the average value of those questions - the higher the value the higher the diffculty of that collection of questions. 

filtered = filter_data(jeopardy_data, ["Song"])
print(filtered["Float Value"].mean())

755.7432432432432


In [21]:
# A function to find the unique answers of a set of data
def get_answer_counts(data):
    return data["Answer"].value_counts()

# Testing the answer count function
print(get_answer_counts(filtered))

Answer
Toni Morrison       8
Solomon             6
Walt Whitman        5
Harvey Fierstein    4
Norman Mailer       4
                   ..
(Henryk) Gorecki    1
Russian             1
Whitman             1
Eliot               1
Maya Angelou        1
Name: count, Length: 239, dtype: int64
